In [1]:
!pip install accelerate datasets evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [3]:
model_name = "sbunlp/fabert"

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

farstail = load_dataset("PNLPhub/FarsTail")
farstail=farstail.class_encode_column("label")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7266 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1564 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1537 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/7266 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1564 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1537 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True)
tokenized_farstail = farstail.map(preprocess_function, batched = True)

Map:   0%|          | 0/7266 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1564 [00:00<?, ? examples/s]

Map:   0%|          | 0/1537 [00:00<?, ? examples/s]

In [7]:
labels = tokenized_farstail["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [8]:
label2id

{'c': 0, 'e': 1, 'n': 2}

In [10]:
import numpy as np
import evaluate
accuracy = evaluate.load('accuracy')
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
		predictions, labels = eval_pred
		predictions = np.argmax(predictions, axis=1)
		accuracy_result = accuracy.compute(predictions=predictions, references=labels)['accuracy']
		f1_result = f1.compute(predictions=predictions, references=labels,average='macro')['f1']
		return {"accuracy":accuracy_result,"f1":f1_result}

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id),label2id=label2id, id2label=id2label)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="tmp/farstail",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,  # Load the best model based on the specified metric
    metric_for_best_model="f1",
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_farstail["train"],
    eval_dataset=tokenized_farstail["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.535647,0.789850,0.788227
2,0.813600,0.458057,0.830839,0.830234
3,0.453500,0.538106,0.824984,0.826519
4,0.296800,0.653059,0.830839,0.832100
5,0.203300,0.788814,0.831490,0.832274
6,0.135400,0.856472,0.840599,0.840417
7,0.069200,0.894796,0.837345,0.837743


TrainOutput(global_step=3185, training_loss=0.3136252114312428, metrics={'train_runtime': 1121.9343, 'train_samples_per_second': 45.334, 'train_steps_per_second': 2.839, 'total_flos': 2878507040349780.0, 'train_loss': 0.3136252114312428, 'epoch': 7.0})

In [27]:
validation_results = trainer.evaluate()

In [28]:
print(validation_results)

{'eval_loss': 0.8564716577529907, 'eval_accuracy': 0.8405985686402082, 'eval_f1': 0.8404165451945778, 'eval_runtime': 8.1149, 'eval_samples_per_second': 189.406, 'eval_steps_per_second': 11.953, 'epoch': 7.0}


In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_farstail["train"],
    eval_dataset=tokenized_farstail["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
test_results = trainer.evaluate()

In [30]:
print(test_results)

{'eval_loss': 0.9156791567802429, 'eval_accuracy': 0.840153452685422, 'eval_f1': 0.8404217394613999, 'eval_runtime': 8.3166, 'eval_samples_per_second': 188.057, 'eval_steps_per_second': 11.784}


## Pipeline

In [20]:
from transformers import pipeline

# classifier = pipeline("text-classification", model = model,tokenizer = tokenizer) # cpu
classifier = pipeline("text-classification", model = model,tokenizer = tokenizer,device=0) # gpu

some examples from the test set

In [21]:
text1="پدیدارشناسی یا پدیده‌شناسی (فنومنولوژی) فلسفهٔ مطالعهٔ ساختارِ تجربه یا آگاهی است. این جنبش فلسفی در ابتدای سدهٔ بیستم میلادی توسط ادموند هوسرل پایه‌گذاری شد."
text2="هگل پایه پدیده شناسی یا فنومنولوژی را بنا نهاد."

classifier({'text':text1, 'text_pair': text2})
# contradiction

{'label': 'c', 'score': 0.9967709183692932}

In [24]:
text1 = "محمد اقبال لاهوری یا علامه اقبال، شاعر، فیلسوف، سیاست‌مدار و متفکر مسلمان پاکستانی بود که اشعار زیادی نیز به زبان‌های فارسی و اردو سروده‌است. اقبال در این کشور به‌طور رسمی «شاعر ملی» خوانده می‌شود."
text2= "اقبال لاهوری، شاعر ملی پاکستان است."
classifier({'text':text1, 'text_pair': text2})
# entailment


{'label': 'e', 'score': 0.9934316277503967}